#Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

#Function definitions

In [2]:
def create_lagged_features(df, N=7):
    # List to store the lagged dataframes
    lagged_dfs = [df]

    # Create dataframes with N lags per feature
    for n in range(1, N+1):
        lagged_df = df[df.columns[1:]].shift(n)
        lagged_df.columns = [f"{col}_lag{n}" for col in lagged_df.columns]
        lagged_dfs.append(lagged_df)

    # Concatenate the lagged dataframes
    df_lagged = pd.concat(lagged_dfs, axis=1)

    # Drop NaNs
    df_lagged = df_lagged.dropna().reset_index(drop=True)

    return df_lagged

In [3]:
def iterative_prediction(models, df, df_lagged, M=7, N=7):
    # Initialize list to store predictions
    predictions = []

    # Predict for [0,N] days ahead iteratively
    for i in range(len(df_lagged)):
        current_data = df_lagged.iloc[i].copy()
        for m in range(M):
            # Extract features for prediction (only the lagged features)
            features = current_data.filter(like='_lag').values.reshape(1, -1)

            # Initialize directory to store predictions per feature
            predicted_values = {}

            # Make prediction for each variable
            for col, model in models.items():
                predicted_values[col] = model.predict(features)[0]

            # Update the lags
            for col in df.columns[1:]:
                for n in range(N, 0, -1):
                    if n == 1:
                        current_data[f"{col}_lag{n}"] = predicted_values[col]
                    else:
                        current_data[f"{col}_lag{n}"] = current_data[f"{col}_lag{n-1}"]

            # Store the prediction
            predictions.append(predicted_values)

    return pd.DataFrame(predictions)

In [4]:
def compute_mae(model_predicted, data_lagged, N=7):
  # Initialize absolute error dataframe
  error = pd.DataFrame(columns=[f"{col}_ahead{i}" for col in model_predicted.columns for i in range(1, N+1)],
                        index=data_lagged.index[:-N])

  # Compute absolute error for each prediction
  for col in model_predicted.columns:
    for i in range(len(data_lagged) - N):
      for j in range(7):
        error[f"{col}_ahead{j+1}"][i] = abs(model_predicted[f"{col}"][7*i+j] - data_lagged[f"{col}"][i+j])

  # Compute MAE for every target for every [1,N] day ahead
  mae = error.mean().to_frame().T

  # Initialize dictionary to store dataframes for the MAEs of each target
  mae_dict = {}

  # Store MAE dataframes in dictionary as distinct dataframes for each variable
  for ref_col in model_predicted.columns:
      columns = [col for col in mae.columns if col.startswith(ref_col + "_")]
      if columns:
          mae_dict[ref_col] = mae[columns]

  return mae_dict

# Loading Dataset

In [5]:
# Load CSV
hourly = pd.read_csv("/content/drive/MyDrive/Monterrey.csv")
# Format the date
hourly['Fecha'] =  pd.to_datetime(hourly['Fecha'], format='%m/%d/%y %H:%M')
# Drop unused weather variables
hourly = hourly.drop(columns=['DirRafaga', 'DirViento', 'RelHum', 'RapRafaga'])
# Set timestamp as index
hourly.set_index('Fecha', inplace=True)

In [6]:
# Resample the data to daily frequency
data = hourly.resample('D').agg({'Temp': ['mean', 'min', 'max'],
                                 'PresBarometr': ['mean'],
                                 'RadSolar': ['mean', 'max'],
                                 'RapViento': ['mean']})

# Flatten the column names
data.columns = [f'{col[0]}_{col[1]}' for col in data.columns]

# Reset the index
data.reset_index(inplace=True)

# Drop NaNs
data = data.dropna()

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4411 entries, 6 to 6513
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Fecha              4411 non-null   datetime64[ns]
 1   Temp_mean          4411 non-null   float64       
 2   Temp_min           4411 non-null   float64       
 3   Temp_max           4411 non-null   float64       
 4   PresBarometr_mean  4411 non-null   float64       
 5   RadSolar_mean      4411 non-null   float64       
 6   RadSolar_max       4411 non-null   float64       
 7   RapViento_mean     4411 non-null   float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 310.1 KB


# Feature Engineering

In [8]:
# Create df with lagged features
data_lagged = create_lagged_features(data, N=7)
data_lagged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 57 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Fecha                   4404 non-null   datetime64[ns]
 1   Temp_mean               4404 non-null   float64       
 2   Temp_min                4404 non-null   float64       
 3   Temp_max                4404 non-null   float64       
 4   PresBarometr_mean       4404 non-null   float64       
 5   RadSolar_mean           4404 non-null   float64       
 6   RadSolar_max            4404 non-null   float64       
 7   RapViento_mean          4404 non-null   float64       
 8   Temp_mean_lag1          4404 non-null   float64       
 9   Temp_min_lag1           4404 non-null   float64       
 10  Temp_max_lag1           4404 non-null   float64       
 11  PresBarometr_mean_lag1  4404 non-null   float64       
 12  RadSolar_mean_lag1      4404 non-null   float64 

In [9]:
# Split the data into features and targets
X = data_lagged.filter(like='_lag') # Only lags are features
y = data_lagged[data.columns[1:]] # Current weather variables are targets

# RF Model

In [10]:
# Dictionary to store trained models for each weather variable
rf_models = {}

# Train models
for col in y.columns:
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X, y[col])
    rf_models[col] = rf

rf_models

{'Temp_mean': RandomForestRegressor(random_state=42),
 'Temp_min': RandomForestRegressor(random_state=42),
 'Temp_max': RandomForestRegressor(random_state=42),
 'PresBarometr_mean': RandomForestRegressor(random_state=42),
 'RadSolar_mean': RandomForestRegressor(random_state=42),
 'RadSolar_max': RandomForestRegressor(random_state=42),
 'RapViento_mean': RandomForestRegressor(random_state=42)}

In [11]:
%%capture
# Predict weather variables for up to N days ahead using trained models
rf_predicted = iterative_prediction(rf_models, data, data_lagged)

In [12]:
# Compute the MAE per variable per day ahead predicted
rf_mae = compute_mae(rf_predicted, data_lagged)
for key, df in rf_mae.items():
    print(f"{key}")
    display(df)

Temp_mean


,Temp_mean_ahead1,Temp_mean_ahead2,Temp_mean_ahead3,Temp_mean_ahead4,Temp_mean_ahead5,Temp_mean_ahead6,Temp_mean_ahead7
0,0.647726,1.275094,1.821839,2.212985,2.489005,2.679632,2.79894


Temp_min


,Temp_min_ahead1,Temp_min_ahead2,Temp_min_ahead3,Temp_min_ahead4,Temp_min_ahead5,Temp_min_ahead6,Temp_min_ahead7
0,0.522078,1.025448,1.481393,1.83799,2.090992,2.25865,2.368934


Temp_max


...

,RadSolar_mean_ahead1,RadSolar_mean_ahead2,RadSolar_mean_ahead3,RadSolar_mean_ahead4,RadSolar_mean_ahead5,RadSolar_mean_ahead6,RadSolar_mean_ahead7
0,19.423235,35.647257,48.008452,56.083401,61.461165,65.742173,68.873565


RadSolar_max


,RadSolar_max_ahead1,RadSolar_max_ahead2,RadSolar_max_ahead3,RadSolar_max_ahead4,RadSolar_max_ahead5,RadSolar_max_ahead6,RadSolar_max_ahead7
0,46.636083,80.86727,106.353743,126.992148,141.959591,152.779139,159.991542


RapViento_mean


,RapViento_mean_ahead1,RapViento_mean_ahead2,RapViento_mean_ahead3,RapViento_mean_ahead4,RapViento_mean_ahead5,RapViento_mean_ahead6,RapViento_mean_ahead7
0,0.613407,1.081516,1.420991,1.646674,1.792883,1.892282,1.957406


# GBM Model

In [13]:
# Dictionary to store trained models for each weather variable
gbm_models = {}

# Train models
for col in y.columns:
    gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    gbm.fit(X, y[col])
    gbm_models[col] = gbm

gbm_models

{'Temp_mean': GradientBoostingRegressor(random_state=42),
 'Temp_min': GradientBoostingRegressor(random_state=42),
 'Temp_max': GradientBoostingRegressor(random_state=42),
 'PresBarometr_mean': GradientBoostingRegressor(random_state=42),
 'RadSolar_mean': GradientBoostingRegressor(random_state=42),
 'RadSolar_max': GradientBoostingRegressor(random_state=42),
 'RapViento_mean': GradientBoostingRegressor(random_state=42)}

In [14]:
%%capture
# Predict weather variables for up to N days ahead using trained models
gbm_predicted = iterative_prediction(gbm_models, data, data_lagged)

In [15]:
# Compute the MAE per variable per day ahead predicted
gbm_mae = compute_mae(gbm_predicted, data_lagged)
for key, df in gbm_mae.items():
    print(f"{key}")
    display(df)

Temp_mean


,Temp_mean_ahead1,Temp_mean_ahead2,Temp_mean_ahead3,Temp_mean_ahead4,Temp_mean_ahead5,Temp_mean_ahead6,Temp_mean_ahead7
0,1.494785,2.179674,2.471735,2.635132,2.728964,2.808629,2.84737


Temp_min


,Temp_min_ahead1,Temp_min_ahead2,Temp_min_ahead3,Temp_min_ahead4,Temp_min_ahead5,Temp_min_ahead6,Temp_min_ahead7
0,1.226078,1.781784,2.083402,2.270631,2.396849,2.462708,2.519815


Temp_max


...

,RadSolar_mean_ahead1,RadSolar_mean_ahead2,RadSolar_mean_ahead3,RadSolar_mean_ahead4,RadSolar_mean_ahead5,RadSolar_mean_ahead6,RadSolar_mean_ahead7
0,46.525014,57.430695,62.180436,64.871132,67.1783,69.596795,71.984423


RadSolar_max


,RadSolar_max_ahead1,RadSolar_max_ahead2,RadSolar_max_ahead3,RadSolar_max_ahead4,RadSolar_max_ahead5,RadSolar_max_ahead6,RadSolar_max_ahead7
0,104.899861,124.345103,135.038165,143.598639,151.440902,158.597048,166.23313


RapViento_mean


,RapViento_mean_ahead1,RapViento_mean_ahead2,RapViento_mean_ahead3,RapViento_mean_ahead4,RapViento_mean_ahead5,RapViento_mean_ahead6,RapViento_mean_ahead7
0,1.466014,1.730011,1.857688,1.92858,1.970334,2.004565,2.036238


# SVM Model

In [16]:
# Dictionary to store trained models for each weather variable
svm_models = {}

# Train models
for col in y.columns:
    svm = SVR(kernel='rbf', C=1.0)
    svm.fit(X, y[col])
    svm_models[col] = svm

svm_models

{'Temp_mean': SVR(),
 'Temp_min': SVR(),
 'Temp_max': SVR(),
 'PresBarometr_mean': SVR(),
 'RadSolar_mean': SVR(),
 'RadSolar_max': SVR(),
 'RapViento_mean': SVR()}

In [17]:
%%capture
# Predict weather variables for up to N days ahead using trained models
svm_predicted = iterative_prediction(svm_models, data, data_lagged)

In [18]:
# Compute the MAE per variable per day ahead predicted
svm_mae = compute_mae(svm_predicted, data_lagged)
for key, df in svm_mae.items():
    print(f"{key}")
    display(df)

Temp_mean


,Temp_mean_ahead1,Temp_mean_ahead2,Temp_mean_ahead3,Temp_mean_ahead4,Temp_mean_ahead5,Temp_mean_ahead6,Temp_mean_ahead7
0,3.279365,3.511245,3.634292,3.714951,3.800166,3.903681,4.025467


Temp_min


,Temp_min_ahead1,Temp_min_ahead2,Temp_min_ahead3,Temp_min_ahead4,Temp_min_ahead5,Temp_min_ahead6,Temp_min_ahead7
0,3.241406,3.34953,3.447236,3.526833,3.608531,3.703767,3.807409


Temp_max


...

,RadSolar_mean_ahead1,RadSolar_mean_ahead2,RadSolar_mean_ahead3,RadSolar_mean_ahead4,RadSolar_mean_ahead5,RadSolar_mean_ahead6,RadSolar_mean_ahead7
0,66.430825,70.353905,71.993035,73.331076,74.730803,76.246283,77.949821


RadSolar_max


,RadSolar_max_ahead1,RadSolar_max_ahead2,RadSolar_max_ahead3,RadSolar_max_ahead4,RadSolar_max_ahead5,RadSolar_max_ahead6,RadSolar_max_ahead7
0,134.299979,140.637464,144.447349,147.984084,151.707617,155.874929,160.993036


RapViento_mean


,RapViento_mean_ahead1,RapViento_mean_ahead2,RapViento_mean_ahead3,RapViento_mean_ahead4,RapViento_mean_ahead5,RapViento_mean_ahead6,RapViento_mean_ahead7
0,2.074468,2.114735,2.163394,2.191165,2.213806,2.24128,2.267826
